# Librerias

In [ ]:
import numpy as np
import pandas as pd
from random import shuffle

# Game setup

## Tablero

Definimos el tablero utilizando ```numpy```

In [ ]:
board = np.zeros((6,7))
board

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

## Movimientos

Definimos una función para saber si un movimiento es válido o no

In [ ]:
def is_valid(board, y):
  if(board[0][y] != 0):
    return False
  return True

Ahora creamos una función para definir si algún jugador ha ganado el juego. Esta función reconoce movimientos ganadores dentro del tablero

Retorna el ganador
- Retorna 1 si el jugador A es el ganador
- Retorna 2 si el jugador B es el ganador
- Retorna 0 si es un empate


In [ ]:
def end_game(board):
  rows = len(board)
  cols = len(board[0])

  # Busca las victorias horizontales
  for row in range(rows):
      for col in range(cols - 3):
          if all(board[row][col+i] == 1 if col+i < cols else False for i in range(4)):
              return 1
          if all(board[row][col+i] == 2 if col+i < cols else False for i in range(4)):
              return 2

  # Busca las victorias verticales
  for col in range(cols):
      for row in range(rows - 3):
          if all(board[row+i][col] == 1 if row+i < rows else False for i in range(4)):
              return 1
          if all(board[row+i][col] == 2 if row+i < rows else False for i in range(4)):
              return 2
  # Busca las victorias en diagonal (de izquierda arriba a derecha abajo)
  for row in range(rows - 3):
      for col in range(cols - 3):
          if all(board[row+i][col+i] == 1 if col+i < cols and row+1 < rows else False for i in range(4)):
            return 1
          if all(board[row+i][col] == 2 if row+i < rows else False for i in range(4)):
            return 2
  # Busca las victorias en diagonal (de izquierda baja a derecha arriba)
  for row in range(3, rows):
      for col in range(cols - 3):
          if all(board[row-i][col+i] == 1 if col+i < cols else False for i in range(4)):
              return 1
          if all(board[row+i][col] == 2 if row+i < rows else False for i in range(4)):
            return 2

  return 0

## Movimiento

La función ```move``` registra el movimiento del jugador proveido dentro del tablero tomando en cuanta la lógica de connect-4

In [ ]:
def move(board, y, player):
  ROWS = len(board)
  COLS = len(board[0])
  if y > COLS-1 or y < 0:
    raise 'Non valid'

  if board[0][y] != 0:
    raise 'Non valid'

  if not is_valid(board, y):
    raise 'Non valid'

  for idx, row in enumerate(board):
    if row[y] != 0:
      board[idx-1][y] = player
      break
    if idx == ROWS-1:
      board[ROWS-1][y] = player
  return board

## Posiciones válidas

La función ```get_valid_locations``` devuelve las posiciones disponibles para lanzar

In [ ]:
def get_valid_locations(board):
    valid_locations = []
    for col in range(len(board[0])):
        if is_valid(board, col):
            valid_locations.append(col)
    return valid_locations

Devuelve un arreglo con las columnas disponibles para realizar un movimiento

In [ ]:
get_valid_locations(board)

[0, 1, 2, 3, 4, 5, 6]

## Reconocimiento del final del juego

In [ ]:
def terminal_move(board):
  return end_game(board) != 0 or len(get_valid_locations(board)) == 0

# Algoritmo

Para resolver este problema se utiliza el algoritmo min max junto a la poda alfa beta. En este caso, se toma al jugador A (representado por 1) como el max y el jugador B (representado por 2) como el min

# Heurísticas y sistema de recompensas

Para apoyar al algoritmo y lograr que el juego se dé de la manera más "inteligente" posible. Se desarrollaron diferentes estrategias

Primeramente se definió una función llamada ```find_all_winning_moves``` que se encarga de identificar si el jugador proveido está a una pieza de ganar

In [ ]:
def find_all_winning_moves(board, player):
    winning_moves = []

    # Registra en dirección horizontal
    for row in range(6):
        for col in range(4):
            sequence = board[row, col:col + 4]
            if np.count_nonzero(sequence == player) == 3 and np.count_nonzero(sequence == 0) == 1:
                empty_index = np.where(sequence == 0)[0][0]
                winning_moves.append((row, col + empty_index))

    # Registra en dirección vertical
    for row in range(3):
        for col in range(7):
            sequence = board[row:row + 4, col]
            if np.count_nonzero(sequence == player) == 3 and np.count_nonzero(sequence == 0) == 1:
                empty_index = np.where(sequence == 0)[0][0]
                winning_moves.append((row + empty_index, col))
    # Registra en dirección diagonal de izquierda arriba a derecha abajo
    for row in range(3):
        for col in range(4):
            sequence = np.array([board[row + i, col + i] for i in range(4)])
            if np.count_nonzero(sequence == player) == 3 and np.count_nonzero(sequence == 0) == 1:
                empty_index = np.where(sequence == 0)[0][0]
                winning_moves.append((row + empty_index, col + empty_index))

    # Registra en dirección diagonal de izquierda abajo a derecha arriba
    for row in range(3, 6):
        for col in range(4):
            sequence = np.array([board[row - i, col + i] for i in range(4)])
            if np.count_nonzero(sequence == player) == 3 and np.count_nonzero(sequence == 0) == 1:
                empty_index = np.where(sequence == 0)[0][0]
                winning_moves.append((row - empty_index, col + empty_index))

    final = []
    for move in winning_moves:
      if move[0] + 1 < 6:
        if board[move[0]+1][move[1]] != 0:
          final.append(move[1])
      else:
        final.append(move[1])
    return final

Por otro lado, se realizó una función para darle recompensas al algoritmo que se realizó un movimiento satisfactorio

In [ ]:
def evaluate_window(window, piece):
    score = 0
    # Cambia el puntaje dependiendo del turno
    opp_piece = 2
    if piece == 2:
        opp_piece = 1

    # Prioriza victorias
    if window.count(piece) == 4:
        score += 100
    # Da puntos por conectar 3 piezas
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 5
    # Da puntos por conectar 2 piezas
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 2
    # Bloquea la victoria del oponente
    if window.count(opp_piece) == 3 and window.count(0) == 1:
        score -= 4

    return score

In [ ]:
def score_position(board, piece):
    score = 0

    # Da puntaje para la columna central
    centre_array = [int(i) for i in list(board[:, len(board[0]) // 2])]
    centre_count = centre_array.count(piece)
    score += centre_count * 3

    # Da puntaje en posiciones horizontales
    for r in range(len(board)):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(len(board[0]) - 3):
            # Create a horizontal window of 4
            window = row_array[c:c + 4]
            score += evaluate_window(window, piece)

    # Da puntaje en posiciones verticales
    for c in range(len(board[0])):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(len(board) - 3):
            # Create a vertical window of 4
            window = col_array[r:r + 4]
            score += evaluate_window(window, piece)

    # Da puntaje en posiciones diagonales (arriba abajo)
    for r in range(len(board) - 3):
        for c in range(len(board[0]) - 3):
            # Create a positive diagonal window of 4
            window = [board[r + i][c + i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Da puntaje en posiciones diagonales (abajo arriba)
    for r in range(len(board) - 3):
        for c in range(len(board[0]) - 3):
            # Create a negative diagonal window of 4
            window = [board[r + 3 - i][c + i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

Ahora se realiza el algoritmo al cual se le dan los parámetros


*   El tablero
*   La profundidad máxima
*   Alfa
*   Beta
*   Si se está maximizando o minimizando
*   Si es el primer movimiento o no

Ahora bien, el algoritmo tiene dos modos de funcionamiento.
1. Se le provee un movimiento inicial al algoritmo y el algoritmo evalua a partir de ese movimiento (Se define la variable global ```MOVE``` y se agrega el parámetro ```first_move=True```)
2. No se provee el primer movimiento


In [ ]:
MOVE = 6

In [ ]:
def minimax(board, depth, alpha, beta, maximisingPlayer, first_move=True):
    # Se encuentran los movimientos ganadores para el jugador actual
    winning = find_all_winning_moves(board, 1 if maximisingPlayer else 2)
    file.write(f'alpha: {alpha} \n beta: {beta} \n board: {board} \n ')
    # Se encuentran los movimientos de ganadores para el oponente
    block = find_all_winning_moves(board, 2 if maximisingPlayer else 1)
    winning.extend(block)
    # Se agregan los movimientos a un arreglo dónde se le da prioridad a ganar antes de bloquear
    moves = winning
    if len(moves) == 0:
      # Si no hay movimientos ganadores o de bloqueo se consiguen las posiciones disponibles para mover y se elige un movimiento aleatorio
      valid_locations = get_valid_locations(board)
      shuffle(valid_locations)
      moves = valid_locations

    # Se evalua si ya se acabó el juego
    is_terminal = terminal_move(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            # Se le da un puntaje demasiado alto a la victoria del jugador 1
            if end_game(board) == 1:
                return (None, 9999999)
            # Se le da un puntaje demasiado bajo a la victoria del jugador 2
            elif end_game(board) == 2:
                return (None, -9999999)
            else:  # Hubo empate
                return (None, 0)
        # Retorna el puntaje del bot
        else:
            return (None, score_position(board, 1))

    if maximisingPlayer:
        # Conducta al maximizar
        value = -9999999
        column = MOVE
        if first_move:
          # Crea una copia del tablero
          b_copy = board.copy()
          # Hace un movimiento en la columna inicial
          b_copy = move(b_copy, column, 1)
          file.write(f'move on column: {column} \n')
          new_score = minimax(b_copy, depth - 1, alpha, beta, False, False)[1]
          if new_score > value:
                value = new_score
          alpha = max(alpha, value)
          return column, value
        else:
          for col in moves:
              # Create a copy of the board
              b_copy = board.copy()
              # Drop a piece in the temporary board and record score
              b_copy = move(b_copy, col, 1)
              file.write(f'move on column: {col} \n')
              new_score = minimax(b_copy, depth - 1, alpha, beta, False, False)[1]
              if new_score > value:
                  value = new_score
              alpha = max(alpha, value)
              if alpha >= beta:
                  break
          return column, value

    else:
        value = 9999999
        # Conducta al minimizar
        column = MOVE
        if first_move:
          b_copy = board.copy()
          b_copy = move(b_copy, column, 2)
          file.write(f'move on column: {column} \n')
          new_score = minimax(b_copy, depth - 1, alpha, beta, True, False)[1]
          if new_score < value:
                  value = new_score
                  column = col
          beta = min(beta, value)
          return column, value
        else:
          for col in moves:
              b_copy = board.copy()
              b_copy = move(b_copy, col, 2)
              file.write(f'move on column: {col} \n')
              new_score = minimax(b_copy, depth - 1, alpha, beta, True, False)[1]
              if new_score < value:
                  value = new_score
                  column = col
              beta = min(beta, value)
              if alpha >= beta:
                  break
        return column, value

### Creación del archivo de muestra

Se crea un archivo para mostrar las diferentes variables y estados. Al final del archivo se encuentra el veredicto del algoritmo

In [ ]:
file = open('/content/game.txt', 'w')

Se inicializa alfa y beta en 0, junto al tablero y el parametro de maximización como ```True```

In [ ]:
result = minimax(board, 42, 0, 0, True)
result

(6, 9999999)

Se agrega el resultado en el archivo a partir del resultado de la función

In [ ]:
if result[1] == 9999999:
  file.write('Probably will win player 1')
elif result[1] == -9999999:
  file.write('Probably will win player 2')
else:
  file.write('Probably a tie')
file.close()

El archivo con las variables, el resultado y los estados se encuentra en la raiz del directorio local del notebook en google colab. Si se usa otro medio, configurar la ruta deseada desde la apertura del archivo.